In [1]:
#test for 给某一层赋值
import os
import pyro
import torch
import pickle
import time

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import pyro.distributions as dist

from tqdm.auto import trange
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO, Predictive
from pyro.distributions import constraints
from pyro.infer.autoguide import AutoDiagonalNormal

path = os.getcwd()
os.chdir("../../")
cwd = os.getcwd()
print(cwd)
from src.dgp_with_init.deep_layer import DeepGP, DeepGPNoBias, DeepEnsembleGP
# os.chdir(path)
# from src.dgp_rff.get_variable import getvalue

c:\Users\yuanq\OneDrive\Desktopold\SB\research\DGPII\program\DGP-RFF-main


In [2]:
import torch
torch.cuda.is_available()
# trouble shoot see this link:https://stackoverflow.com/questions/77068908/how-to-install-pytorch-with-cuda-support-on-windows-11-cuda-12-no-matching

True

In [3]:
start = time.perf_counter()
torch.cuda.current_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
# cuda = torch.device('cuda') 

In [4]:
var1= torch.FloatTensor([1.0,2.0,3.0]).cuda()

In [5]:
var1.device

device(type='cuda', index=0)

In [6]:
torch.cuda.memory_allocated()

512

  # Next step

0. Get familiar with the coding structure
1. CPU -> GPU
2. Last step without bias
3. Figure out how they train
4. How to access the posterior mean and std/scale from the model
5. Learn pickle: numpy.ndarray, png
6. ....
7. construct a DeepGP class

In [7]:
# Set random seed for reproducibility
np.random.seed(1)

# Read data
# cwd = os.getcwd()
# print(cwd)

X_train_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_X_train.txt")
X_test_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_X_test.txt")
Y_train_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_Y_train.txt")
Y_test_path = os.path.join(cwd, "folds", "synthetic_1_fold_1_Y_test.txt")

x_obs = np.loadtxt(X_train_path)
y_obs = np.loadtxt(Y_train_path)
x_val = np.loadtxt(X_test_path)
y_val = np.loadtxt(Y_test_path)

# Set plot limits and labels
xlims = [-0.2, 0.2]

# The X and Y have to be at least 2-dim
x_train = torch.from_numpy(x_obs).float().reshape(-1,1)
y_train = torch.from_numpy(y_obs).float()
x_test = torch.from_numpy(x_val).float().reshape(-1,1)
y_test = torch.from_numpy(y_val).float()

In [8]:
x_train = x_train.cuda()


In [9]:
y_train = y_train.cuda()
x_test = x_test.cuda()
y_test = y_test.cuda()

In [10]:

x_train.device

device(type='cuda', index=0)

In [11]:


  # X = [
#     [1,2,3],
#     [4,5,6,7,8]
#     ]
# X_path = os.path.join(cwd, "folds", "test.pickle")
# with open(X_path, 'wb') as f:
#     pickle.dump(X, f)

In [12]:
# with open(X_path, 'rb') as f:
#     X_read = pickle.load(f)

In [65]:
class ModelDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[5,10], **kwargs):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGP(dim_list, J_list, **kwargs)
        self.model.to('cuda')
        self.kwargs = kwargs

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise #original 0.5 loc
        if 'init_scale' in self.kwargs:
            init_w = self.kwargs['init_scale']
            # print("init_w", init_w.mean.shape, init_w.var.shape)
            scale = pyro.sample("sigma", dist.Gamma(init_w.mean, init_w.var)).expand(self.out_dim) 
            print("scale initialization success!")
        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [22]:
class ModelEnsembleDGP(PyroModule):
    def __init__(self, dim_list = [1,1,1], J_list=[50,10]):
        super().__init__()
        self.out_dim = dim_list[-1]
        self.model = DeepEnsembleGP(dim_list, J_list,2)
        self.model.to('cuda')
        
    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        # mu = self.linear(x1).squeeze()
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [23]:
class ModelNoBiasDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGPNoBias(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample( "obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)
            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [24]:
print([x_train.shape[1],10,y_train.shape[1]])

[1, 10, 6]


In [25]:
class InitParams:
    def __init__(self, mean: float, var: float) -> None:
        self.mean = mean
        self.var = var

    def __repr__(self) -> str:
        return f"InitParams(mean={self.mean}, var={self.var})"


In [75]:
# load_variable.py
import pickle
# with open('loc_test.pkl', 'rb') as file:
loc = pickle.load(open('loc_test.pkl', 'rb'))
scale = pickle.load(open('scale_test.pkl', 'rb'))
for i in range(len(loc)):
    print(np.shape(loc[i]))

dim_list=[x_train.shape[1],5,y_train.shape[1]]
# J_list=[4,4]
layer_num = 2
in_dim_list = [1,5] 
out_dim_list = [5,6]
J_list = [4,4]
print(dim_list, J_list)

params = {}
for i in range(layer_num):
    params[f'init_1st_w{i}'] = InitParams(loc[3*i+0].reshape(J_list[i], in_dim_list[i]), scale[3*i+0].reshape(J_list[i], in_dim_list[i]))
    params[f'init_2nd_w{i}'] = InitParams(loc[3*i+1].reshape(out_dim_list[i], 2*J_list[i]), scale[3*i+1].reshape(out_dim_list[i], 2*J_list[i]))
    params[f'init_2nd_b{i}'] = InitParams(loc[3*i+2], scale[3*i+2])
# params['init_scale'] = InitParams(loc[-1][0].view(1,1), scale[-1][0].view(1,1))

for name, param in params.items():
    print(name, param)

torch.Size([4])
torch.Size([40])
torch.Size([5])
torch.Size([20])
torch.Size([48])
torch.Size([6])
torch.Size([1])
[1, 5, 6] [4, 4]
init_1st_w0 InitParams(mean=tensor([[ 0.0064],
        [ 0.1947],
        [-0.5861],
        [ 0.0297]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>), var=tensor([[0.1002],
        [0.0998],
        [0.1000],
        [0.0998]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>))
init_2nd_w0 InitParams(mean=tensor([[0.8757, 0.8580, 0.4332, 1.0289, 0.8774, 0.8594, 0.7523, 1.7293],
        [0.4680, 0.9804, 0.9377, 0.9663, 0.8830, 1.3790, 1.0478, 0.7780],
        [0.6568, 0.9370, 0.8791, 1.3102, 0.5932, 0.9398, 1.1613, 1.2651],
        [0.9869, 0.9645, 0.8653, 0.5242, 0.6846, 0.7664, 1.2200, 0.8917],
        [1.2042, 0.5465, 0.8870, 0.9336, 1.0041, 0.8777, 0.9700, 0.3611]],
       device='cuda:0', grad_fn=<ReshapeAliasBackward0>), var=tensor([[0.1001, 0.0998, 0.0998, 0.1001, 0.0999, 0.1002, 0.1002, 0.1000],
        [0.1000, 0.0998, 0.1002, 0.1002, 0.1001, 

In [76]:
#model = Model(in_dim_list=[x_train.shape[1],10], out_dim_list=[10,y_train.shape[1]], J_list=[50,10])
mymodel = ModelDGP(dim_list=[x_train.shape[1],5,y_train.shape[1]], J_list=[4,4], **params)
print(x_train.shape)
print(y_train.shape)
mymodel = mymodel.to('cuda')
#报错： tensor([[ 1.2223, -0.1403, -0.4599,  0.0344,  0.8647,  1.4247, -0.3386,  1.0720],
#         [ 2.2278, -0.5137, -1.5127,  0.8181, -0.9370, -1.1531, -1.0935,  0.0875],
#         [-2.0547,  0.3056,  0.8607, -0.2785, -0.5333,  2.3363,  0.6335,  0.5716],
#         [ 0.9926, -0.1179, -0.5080, -0.6154,  0.2463, -1.2726, -0.7783, -0.6244],
#         [ 0.4254,  0.6795,  0.2867, -0.3228,  0.1220, -0.3774, -1.1066,  0.1976]],
#        device='cuda:0')

1st layer w initialization success! torch.Size([4, 1])
2nd layer w initialization success! torch.Size([5, 8])
2nd layer b initialization success! torch.Size([5])
1st layer w initialization success! torch.Size([4, 5])
2nd layer w initialization success! torch.Size([6, 8])
2nd layer b initialization success! torch.Size([6])
torch.Size([10000, 1])
torch.Size([10000, 6])


In [77]:
mean_field_guide = AutoDiagonalNormal(mymodel)
optimizer = pyro.optim.Adam({"lr": 0.001})

svi = SVI(mymodel, mean_field_guide, optimizer, loss=Trace_ELBO())
pyro.clear_param_store()

num_epochs = 2
progress_bar = trange(num_epochs)

for epoch in progress_bar:
    loss = svi.step(x_train, y_train)
    progress_bar.set_postfix(loss=f"{loss / x_train.shape[0]:.3f}")

  0%|          | 0/2 [00:00<?, ?it/s]

In [54]:
param_store = pyro.get_param_store()
for name, value in param_store.items():
    print(name, pyro.param(name).shape)
    print(f"{name}: {value}")

AutoDiagonalNormal.loc torch.Size([124])
AutoDiagonalNormal.loc: Parameter containing:
tensor([ 0.0064,  0.1947, -0.5861,  0.0297,  0.8757,  0.8580,  0.4332,  1.0289,
         0.8774,  0.8594,  0.7523,  1.7293,  0.4680,  0.9804,  0.9377,  0.9663,
         0.8830,  1.3790,  1.0478,  0.7780,  0.6568,  0.9370,  0.8791,  1.3102,
         0.5932,  0.9398,  1.1613,  1.2651,  0.9869,  0.9645,  0.8653,  0.5242,
         0.6846,  0.7664,  1.2200,  0.8917,  1.2042,  0.5465,  0.8870,  0.9336,
         1.0041,  0.8777,  0.9700,  0.3611, -0.0932, -0.2305, -0.0173, -0.3816,
        -0.0940,  0.3988, -0.2806, -0.2910,  0.3262,  0.4273, -0.0342, -0.4607,
         0.0111, -0.2770, -0.0513,  0.0263, -0.4053,  0.1058, -0.3013, -0.4257,
        -0.4652, -0.4146, -0.2978,  0.4056, -0.3689,  1.1373,  1.0821,  0.8170,
         1.1039,  1.2997,  0.8159,  1.0503,  1.3267,  1.0494,  0.9527,  1.2253,
         1.0690,  0.5916,  0.8241,  0.7259,  1.1143,  0.6808,  1.5258,  1.4105,
         1.3433,  1.1795,  1.0709

In [60]:
layer_num = 2
in_dim_list = [1,5] 
out_dim_list = [5,6]
J_list = [4,4]
len_list = []
for i in range(layer_num):
    len_list.append(J_list[i]*in_dim_list[i])
    len_list.append(out_dim_list[i]*2*J_list[i])
    len_list.append(out_dim_list[i])
len_list.append(1)
len_list

[4, 40, 5, 20, 48, 6, 1]

In [61]:
def split_array_by_lengths(array, lengths):
    result = []
    current_position = 0

    for length in lengths:
        if current_position + length <= len(array):
            result.append(array[current_position:current_position + length])
            current_position += length
        else:
            result.append(array[current_position:])
            break

    return result


In [62]:
loc =param_store["AutoDiagonalNormal.loc"]
print(np.shape(loc))

torch.Size([124])


In [63]:
scale =param_store["AutoDiagonalNormal.scale"]
print(np.shape(scale))


torch.Size([124])


In [64]:
result = split_array_by_lengths(loc, len_list)
result_s = split_array_by_lengths(scale, len_list)
# save_variable.py
import pickle
loc_test = result

# with open('loc_test.pkl', 'wb') as file:
pickle.dump(result, open('loc_test.pkl', 'wb'))
pickle.dump(result_s, open('scale_test.pkl', 'wb'))

In [ ]:
mymodel.parameters()

In [ ]:
predictive = Predictive(mymodel, guide=mean_field_guide, num_samples=500)
preds = predictive(x_test)

In [ ]:
y_pred = preds['obs'].cpu().detach().numpy().mean(axis=0)

In [ ]:
preds['obs'].shape

In [ ]:
for d in range(6):
    plt.plot(x_obs, y_obs[:,d], label="Observation")
    plt.plot(x_test.cpu(), y_pred[:,d], label="Prediction")
    plt.legend()
    plt.show()

In [ ]:
# for i in range(int(len(mymodel.model.layers))):
#     print("Layer", i,":")
#     print("RBF Omega = ", mymodel.model.layers[i].layers_RBF[0].layer.weight)
    #print("Cauchy Omega = ", model.model.layers[2*i].layers_Cauchy.omega)
    #print("Laplacian Omega = ", model.model.layers[2*i].layers_Laplacian.omega)
    #print("W = ", model.model.layers[2*i+1].layer.weight.squeeze())
    #print("bias = ", model.model.layers[2*i+1].layer.bias.squeeze())

In [ ]:
#getvalue(mymodel)

In [ ]:
# # sampled Omega: 
# for i in range(int(len(model.model.layers)/2)):
#     print("Layer", 2*i, " and Layer", 2*i+1, ":")
#     print("Omega = ", model.model.layers[2*i].layer.weight.squeeze())
#     print("W = ", model.model.layers[2*i+1].layer.weight.squeeze())
#     print("bias = ", model.model.layers[2*i+1].layer.bias.squeeze())


In [ ]:
# pickle.dump(model, open('DeepRFGP.model.pkl', 'wb'))

In [ ]:
preds['model.layers.0.layer.weight'].squeeze().mean(axis=0)#: Omega estimator

In [ ]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name).shape)

In [ ]:
mymodel.model.layers[0].layer.weight

In [ ]:
end = time.perf_counter()
runtime = end - start
print("Runtime:", runtime)

1 -> 50; 100 -> 6

Omega 1*50  1*50
W     6*100 6*100
bias  6     6
sigma 1     1

In [ ]:
# def model(data):
#   alpha = torch.tensor(6.0)
#   beta = torch.tensor(10.0)
#   pay_probs = pyro.sample('pay_probs', dist.Beta(alpha, beta).expand(3).independent(1))
#   normalized_pay_probs = pay_probs / torch.sum(pay_probs)

#   with pyro.iarange('data_loop', len(data)):
#     pyro.sample('obs', dist.Categorical(probs=normalized_pay_probs), obs=data)

In [ ]:
# def guide(data):
#   alphas = pyro.param('alphas', torch.tensor(6.).expand(3), constraint=constraints.positive)
#   betas = pyro.param('betas', torch.tensor(10.).expand(3), constraint=constraints.positive) 

#   pyro.sample('pay_probs', dist.Beta(alphas, betas).independent(1))